<a href="https://colab.research.google.com/github/roesta07/How-Business-Strategies-can-create-bias/blob/main/strategy_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>